**!!!!!! Derived from code from Chris Bizon**

In [1]:
import requests
import json

In [2]:
#AC_URL = "http://localhost:6380/query"
# AC_URL = "https://answercoalesce.renci.org/query"

# no update on CI as of 20240605, so switch to dev
# AC_URL = "https://genetics-kp.ci.transltr.io/genetics_provider/trapi/v1.5/query"

# using dev KP since no update on CI
AC_URL = "https://translator.broadinstitute.org/genetics_provider/trapi/v1.5/query"


In [3]:
def generate_trapi_query(input_node_type,output_node_type,input_curies,predicate,input_is_subject=True):
    envelope = {"message":{"query_graph": {"nodes":{"input":{},"output":{}}, "edges":{"edge_0":{}}}}}
    input_node = envelope["message"]["query_graph"]["nodes"]["input"]
    input_node["categories"] = [input_node_type]
    input_node["ids"] = ["uuid:1"]
    input_node["member_ids"] = input_curies
    input_node["set_interpretation"]="MANY"
    output_node = envelope["message"]["query_graph"]["nodes"]["output"]
    output_node["categories"] = [output_node_type]
    query_edge = envelope["message"]["query_graph"]["edges"]["edge_0"]
    if input_is_subject:
        query_edge["subject"] = "input"
        query_edge["object"] = "output"
    else:
        query_edge["subject"] = "output"
        query_edge["object"] = "input"
#     query_edge["predicates"]  = [ predicate ]
    #query_edge["knowledge_type"] = "inferred"
    return envelope


In [4]:
def phenotypes_to_disease(phenotypes):
    #Phenotype to Disease
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Disease"
    predicate = "biolink:has_phenotype"
    input_is_subject = "False"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)
    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

def phenotypes_to_gene(phenotypes):
    #Phenotype to Gene
    input_type = "biolink:PhenotypicFeature"
    output_type = "biolink:Gene"
    predicate = "biolink:genetically_associated_with"
    input_is_subject = "True"
    query = generate_trapi_query( input_type, output_type, phenotypes, predicate, input_is_subject)

    # print(json.dumps(query, indent=2))

    response = requests.post( AC_URL, json = query )
    print(response.status_code)
    return response.json()

In [9]:
def print_results(resp, count=10):
    for result in resp["message"]["results"][:count]:
        nb = result["node_bindings"]["output"][0]["id"]

        # bug in trapi result fixed - NO LONGER NEEDED
        # nb = result["node_bindings"]["output"]["id"]
        # score = result['analyses']["score"]
        # bug in trapi result fixed - NO LONGER NEEDED

        score = result['analyses'][0]["score"]
        name = resp["message"]["knowledge_graph"]["nodes"][nb]["name"]
        # print(nb, name, score)

        print("{}\t{}\t{}".format(nb, name, score))

In [8]:
#Leigh Disease?
leigh_phenotypes = ["HP:0000739", #| Anxiety,
                    "HP:0001288", # | Gait disturbance,
                    "HP:0001252", # | Hypotonia,
                    "HP:0001250", # | Seizure,
                    "HP:0000750", # | Delayed speech and language development,
                    "HP:0002378", # | Hand tremor,
                    "HP:0002019", # | Constipation,
                    "HP:0007146"] # | Bilateral basal ganglia lesions

response = phenotypes_to_gene(leigh_phenotypes)
print_results(response)

200
NCBIGene:6323	SCN1A	7.379336506102545
NCBIGene:57084	SLC17A6	6.788339474806989
NCBIGene:767558	LUZP6	5.0185791443519605
NCBIGene:8927	BSN	4.95764208592248
NCBIGene:5080	PAX6	4.951220566645148
NCBIGene:6657	SOX2	4.876013485689922
NCBIGene:85439	STON2	4.738453461149258
NCBIGene:81832	NETO1	4.655261581201219
NCBIGene:9381	OTOF	4.588409942090813
NCBIGene:2263	FGFR2	4.576406057832605


In [11]:
malan_phenotypes = [
    "HP:0000718", # aggression         | 2 yrs - present
    "HP:0000739", #anxiety                | 2 yrs - present
    "HP:0002616", # Aortic root aneurysm HP:0002616 | 10 yrs - present
    "HP:0002019", #constipation (occasional) HP:0002019 |        birth - present
    #occasional bright lime-green stool | 5 yrs - present
    "HP:0000678", #crowded teeth        HP:0000678
    "HP:0000886", # Deformed rib cage HP:0000886 | 6 months - present
    "HP:0002046", #heat intolerance        HP:0002046
    "HP:0002705", #high arched palate        HP:0002705
    "HP:0001290", #hypotonia        HP:0001290 |        birth - present
    "HP:0000737", # irritable        HP:0000737 |        2 yrs - present
    "HP:0001382", # Joint hypermobility HP:0001382 | birth - present
    "HP:0000720", # mood swings        HP:0000720 |        2 yrs - present
    "HP:0012801", # narrow jaw        HP:0012801
    "HP:0010865", # Oppositional defiant disorder HP:0010865 | 2 yrs - present
    #Destructive tendencies | 3 yrs - present
    "HP:0002172", # Postural instability HP:0002172 | birth - present
    "HP:0001250", # seizures        HP:0001250 |        7 yrs - present
    "HP:0001649", # Tachycardia HP:0001649 | 1 yr - present
    "HP:0001337", # tremor        HP:0001337 | in hands 11 yrs - present
    #Vascular skin abnormality (superficial blood vessels that rupture easily) HP:0011276 |  birth - present
    "HP:0001123" # Visual field defect HP:0001123 | birth - present
    # Seasonal allergies | birth - present
    #Food allergy | 4 yrs - present
]

response = phenotypes_to_gene(malan_phenotypes)
print_results(response)

200
NCBIGene:6323	SCN1A	9.137794522919695
NCBIGene:57084	SLC17A6	8.201870463561473
NCBIGene:7042	TGFB2	8.117534161788132
NCBIGene:2200	FBN1	7.441528743758441
NCBIGene:7422	VEGFA	7.266715351815643
NCBIGene:1030	CDKN2B	6.940328049560195
NCBIGene:5728	PTEN	6.7580925357297055
NCBIGene:4016	LOXL1	6.350769734207485
NCBIGene:4318	MMP9	6.308635185861958
NCBIGene:8927	BSN	6.294213627139217


In [10]:
tbcd_phenotypes=[
    "HP:0002098", # | Respiratory distress,
    "HP:0001252", # | Hypotonia,
    "HP:0001250" # | Seizure
]

response = phenotypes_to_gene(tbcd_phenotypes)
print_results(response)


200
NCBIGene:6323	SCN1A	7.026759947689731
NCBIGene:57084	SLC17A6	6.4720157469666075
NCBIGene:767558	LUZP6	4.641834025037843
NCBIGene:8927	BSN	4.561704298324721
NCBIGene:6657	SOX2	4.56075601444945
NCBIGene:85439	STON2	4.3094616312659575
NCBIGene:9381	OTOF	4.201357629454257
NCBIGene:4293	MAP3K9	4.103210578870724
NCBIGene:54715	RBFOX1	4.0776070039933305
NCBIGene:10568	SLC34A2	4.05484826443283


In [ ]:
wdyf_phenotypes = [
    #Global developmental delay
    "HP:0001263",
    #Seizure
    "HP:0001250",
    #Neurodevelopmental delay
    "HP:0012758",
    #Delayed social development
    "HP:0012434"
]

response = phenotypes_to_gene(wdyf_phenotypes)
print_results(response)

200
NCBIGene:6323 SCN1A 7.026759925348748
NCBIGene:57084 SLC17A6 6.4720157207159525
NCBIGene:767558 LUZP6 4.641833985773565
NCBIGene:8927 BSN 4.561704267326607
NCBIGene:6657 SOX2 4.560755986634926
NCBIGene:85439 STON2 4.309461603339728
NCBIGene:9381 OTOF 4.201357604488209
NCBIGene:4293 MAP3K9 4.103210552899331
NCBIGene:54715 RBFOX1 4.07760697422397
NCBIGene:10568 SLC34A2 4.05484823762365


In [ ]:

twogene_phenotypes = [
    #Recurrent fever
    "HP:0001954",
    #hip dysplasia
    "HP:0001385",
    #joint hypermobility
    "HP:0001382",
    #optic disc pallor
    "HP:0000543",
    #recurrent infections
    "HP:0002719",
    #thin corpus callosum
    "HP:0002079",
    #Global developmental delay
    "HP:0001263",
    #Abnormality of vision
    "HP:0000504",
    #Nystagmus
    "HP:0000639",
    #Drooling
    "HP:0002307",
    #Dysphagia
    "HP:0002015",
    #Hypotonia
    "HP:0001252"
    #Eczema
    "HP:0000964",
    #Inability to walk
    "HP:0002540",
    #Septo-optic dysplasia
    "HP:0100842"
    #Postural instability
    "HP:0002172"
]



response = phenotypes_to_gene(twogene_phenotypes)
print_results(response)

200
NCBIGene:55008	HERC6	4.43468515829401e-07
NCBIGene:84109	QRFPR	4.289468767720151e-07
NCBIGene:1488	CTBP2	4.1665933603115006e-07
NCBIGene:2736	GLI2	3.8035523838768527e-07
NCBIGene:259217	HSPA12A	3.7644556633377367e-07
NCBIGene:144165	PRICKLE1	3.6695064848855977e-07
NCBIGene:9	NAT1	3.608068781181273e-07
NCBIGene:7037	TFRC	3.4963638653552273e-07
NCBIGene:9175	MAP3K13	3.462852390607413e-07
NCBIGene:4745	NELL1	3.373488457946577e-07


In [ ]:
ncats_ex_phenotypes = ["HP:0001250","HP:0003593","HP:0010819","HP:0002069","HP:0010818","HP:0002194","HP:0002366","HP:0000750",
                       "HP:0001263","HP:0002072","HP:0000711","HP:0002360","HP:0002360","HP:0001251","HP:0007018","HP:0001249",
                       "HP:0000752","HP:0001252","HP:0002317","HP:0001388","HP:0001382","HP:0001942","HP:0011304","HP:0002269"]

response = phenotypes_to_gene(ncats_ex_phenotypes)
print_results(response, count=20)

200
NCBIGene:51761	ATP8A2	9.277110449004285
NCBIGene:1794	DOCK2	9.020031909316904
NCBIGene:64072	CDH23	8.643628353142566
NCBIGene:627	BDNF	7.922565221088684
NCBIGene:55120	FANCL	7.917125776887522
NCBIGene:2534	FYN	7.474417519366599
NCBIGene:4010	LMX1B	7.419177041901666
NCBIGene:6323	SCN1A	7.362835839884691
NCBIGene:6657	SOX2	7.188111619405844
NCBIGene:6671	SP4	7.15836946094972
NCBIGene:334	APLP2	6.867114602626825
NCBIGene:57084	SLC17A6	6.858865686905159
NCBIGene:259236	TMIE	6.8543138177143925
NCBIGene:64067	NPAS3	6.779646923865877
NCBIGene:222389	BEND7	6.549952178775566
NCBIGene:10439	OLFM1	6.2323775197844595
NCBIGene:23281	MTUS2	6.207324973282342
NCBIGene:8419	BFSP2	6.116639562171504
NCBIGene:100144595	LOC100144595	6.029463589617669
NCBIGene:3097	HIVEP2	5.999007243791686
